<h1>External Dataset Connection<h1>

In [1]:
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = (
    SparkSession.builder.appName("ETL")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/05 22:03:27 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.16 instead (on interface en0)
22/09/05 22:03:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/05 22:03:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3>Add SA2 Labels on Postcode<h3>

In [2]:
joined_data = spark.read.parquet('../data/curated/joined_data.parquet')

In [3]:
correspondence = spark.read.csv('../data/external/sa2_correspondence.csv', header = True)

In [4]:
index = spark.read.csv('../data/external/2019 Locality to 2016 SA2 Coding Index.csv', header = True)

In [5]:
joined_data_sa2 = joined_data.join(index,['postcode'],"left")

In [8]:
df_sa2 = joined_data_sa2.join(correspondence,correspondence.SA2_MAINCODE_2016== joined_data_sa2.SA2_MAINCODE, "outer")

Clean up dataframe by removing columns that are now uneeded

In [9]:
drop_columns = ("LOCALITY_ID", "LOCALITY_NAME", "LOCALITY_TYPE", "STATE", "SA2_MAINCODE", "SA2_NAME", \
               "SA2_MAINCODE_2016", 'SA2_NAME_2016', 'RATIO_FROM_TO', 'INDIV_TO_REGION_QLTY_INDICATOR', \
               'OVERALL_QUALITY_INDICATOR', 'BMOS_NULL_FLAG')
df_sa2 = df_sa2.drop(*drop_columns)
df_sa2 = df_sa2.withColumnRenamed("SA2_CODE_2021", "SA2_code")
df_sa2 = df_sa2.withColumnRenamed("SA2_NAME_2021", "SA2_name")

In [11]:
joined_data.count()

3643266

In [12]:
joined_data_sa2.count()

37619495

There are more counts once joined with SA2 index because some postcodes correspond to multiple SA2 areas

In [13]:
df_sa2.count()

39473912

There are more counts once converted to 2021 SA2